In [1]:
from constants import *
from solver import SolveScheduling
from get_data import *
from train import * 
from models import *
from projectnet import *

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
params = {"n": 24, "c_ramp": 0.4, "gamma_under": 50, "gamma_over": 0.5}
scheduling_solver = SolvePointQP(params)
dist_solver = SolveScheduling(params)

X_train, Y_train, X_test, Y_test, X_train_pt, Y_train_pt, X_test_pt, Y_test_pt = get_data() 

In [3]:
G = scheduling_solver.G[24*2:24*2+23*2,:]
A = torch.cat((G, torch.eye(G.shape[0]).to(DEVICE)), dim=1).float().to(DEVICE)
b = params['c_ramp'] * torch.ones((24 - 1)*2, device=DEVICE).float()

In [4]:
print(A.shape, b.shape)

torch.Size([46, 118]) torch.Size([46])


In [5]:
rounds = 5
projectnet = ProjectNet(A, b, 24, rounds=rounds).to(DEVICE)
train_projectnet(projectnet, Y_train_pt, params, epochs=50, verbose=True) 
torch.save(projectnet.state_dict(), "projectnet.pt")

epoch  0  mean loss:  1.1017354394380863  median  1.0509483218193054 cur:  0.8622184991836548
epoch  1  mean loss:  0.9571446770429611  median  0.941331684589386 cur:  0.6837594509124756
epoch  2  mean loss:  0.7941359728574753  median  0.8484386801719666 cur:  0.7007178068161011
epoch  3  mean loss:  0.7236167746782303  median  0.8097432851791382 cur:  0.5807981491088867
epoch  4  mean loss:  0.6688397830724716  median  0.7655080854892731 cur:  0.5104993581771851
epoch  5  mean loss:  0.582694853246212  median  0.7079532742500305 cur:  0.4150388240814209
epoch  6  mean loss:  0.49595375657081603  median  0.684368908405304 cur:  0.44040292501449585
epoch  7  mean loss:  0.5047060745954514  median  0.662131667137146 cur:  0.44454240798950195
epoch  8  mean loss:  0.5292767044901848  median  0.6324037611484528 cur:  0.46450942754745483
epoch  9  mean loss:  0.4757129240036011  median  0.6030329763889313 cur:  0.36275070905685425
epoch  10  mean loss:  0.4041717004776001  median  0.580399

In [6]:
model_pnet = Net(X_train[:,:-1], Y_train, [200,200]).to(DEVICE)
train_with_pnet(model_pnet, projectnet, X_train_pt, Y_train_pt, params, rounds=rounds, epochs=100, lr=1e-4)
torch.save(model_pnet.state_dict(), "model_pnet.pt")

epoch  0  mean loss:  1.8001214816020086 time 1.940086841583252
epoch  1  mean loss:  1.1464905406420047 time 1.9724336862564087
epoch  2  mean loss:  0.8752684031541531 time 2.0483574072519937
epoch  3  mean loss:  0.6355864295363426 time 2.089836597442627
epoch  4  mean loss:  0.3376831120252609 time 2.106196403503418
epoch  5  mean loss:  0.29158531248569486 time 2.0806414683659873
epoch  6  mean loss:  0.27452532440423966 time 2.0781772136688232
epoch  7  mean loss:  0.26750762611627577 time 2.0620470345020294
epoch  8  mean loss:  0.2611908197402954 time 2.0472243626912436
epoch  9  mean loss:  0.2566872188448906 time 2.0408398389816282
epoch  10  mean loss:  0.2543268083035946 time 2.026160782033747
epoch  11  mean loss:  0.25087924718856813 time 2.013754586378733
epoch  12  mean loss:  0.24938320338726044 time 1.9982930696927583
epoch  13  mean loss:  0.24677032947540284 time 1.9906443527766637
epoch  14  mean loss:  0.24469002157449724 time 1.991917896270752
epoch  15  mean los

KeyboardInterrupt: 

In [7]:
def evaluate_(model):
    with torch.no_grad():
        p = model(X_test_pt)
        costs = []
        for i in range(p.shape[0]):
            d = projectnet(p[i:i+1,:], tol=1e-4).detach()
            c = task_loss(d, Y_test_pt[i:i+1,:], params).mean()
            costs.append(c.item())
    return costs

def evaluate_task(model):
    with torch.no_grad():
        m,s = model(X_test_pt)
        costs = []
        for i in range(m.shape[0]):
            d = dist_solver(m[i:i+1,:],s[i:i+1,:]).detach()
            c = task_loss(d, Y_test_pt[i:i+1,:], params).mean()
            costs.append(c.item())
    return costs

In [8]:
pnet_costs = evaluate_(model_pnet)
print("Test pnet costs:", np.mean(pnet_costs))

Test pnet costs: 0.24061602905555884


In [7]:
task_net = Net2(X_train[:,:-1], Y_train, [200,200]).to(DEVICE)
train_task_net(task_net, params, X_train_pt, Y_train_pt)

epoch: 0 loss: 0.8264765398842948 times: 21.501551866531372
epoch: 1 loss: 0.8239014077754248 times: 21.141249299049377
epoch: 2 loss: 0.822628118689098 times: 21.188942193984985
epoch: 3 loss: 0.8219292561213175 times: 21.07893842458725
epoch: 4 loss: 0.8216588753461838 times: 21.048382806777955
epoch: 5 loss: 0.8211283278465271 times: 20.926042437553406
epoch: 6 loss: 0.8209739810228348 times: 21.096338238034928
epoch: 7 loss: 0.8212644195556641 times: 21.23966908454895
epoch: 8 loss: 0.8215325736999511 times: 21.128661764992607
epoch: 9 loss: 0.8213019561767578 times: 21.128051471710204
epoch: 10 loss: 0.8211598724126816 times: 21.21417381546714
epoch: 11 loss: 0.821280118227005 times: 21.256215155124664
epoch: 12 loss: 0.8214086413383483 times: 21.146953105926514
epoch: 13 loss: 0.8216732960939407 times: 21.103920068059647
epoch: 14 loss: 0.8217941355705262 times: 21.06352915763855
epoch: 15 loss: 0.8217969489097595 times: 21.088600546121597
epoch: 16 loss: 0.8216940528154373 times

Net2(
  (lin): Linear(in_features=149, out_features=24, bias=True)
  (net): Sequential(
    (0): Linear(in_features=149, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=24, bias=True)
  )
)

In [15]:
task_costs = evaluate_task(task_net)

In [16]:
print("Test task costs:", np.mean(task_costs))

Test task costs: 0.8804126808834132
